# Using jupyter-jchannel in a Binder notebook

## Installing

You can use your favorite manager to install the [PyPI package](https://pypi.org/project/jupyter-jchannel/).

```
pip install jupyter-jchannel
```

In a remote notebook, you might need aditional resources to setup a tunnel.

For this specific example, we need to install the [ngrok package](https://pypi.org/project/ngrok/) to setup a [ngrok](https://ngrok.com/) tunnel.

```
pip install ngrok
```

## Starting a server

For basic usage, you only need to import the `jchannel` module.

In this specific example, we also need the `os` module to set an environment variable and the aforementioned `ngrok`.

In [ ]:
import os
import ngrok
import jchannel

A [jchannel Server](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.server.html#jchannel.server.Server) instance runs *alongside* a Jupyter server instead of *over* it. Therefore, it needs its own local address.

In [ ]:
HOST = '127.0.0.1'
PORT = 8889

[Create a ngrok account](https://dashboard.ngrok.com/signup), if you don't have one, copy the token from [this dashboard](https://dashboard.ngrok.com/get-started/your-authtoken), and paste it in the cell below, replacing `YOUR_NGROK_AUTHTOKEN`.

In [ ]:
os.environ['NGROK_AUTHTOKEN'] = 'YOUR_NGROK_AUTHTOKEN'

First, the asynchronous [forward function](https://ngrok.github.io/ngrok-python/module.html#ngrok.forward) creates a ngrok tunnel, using the authentication token in the environment variable, and returns a listener. This listener provides the tunnel URL.

In [ ]:
listener = await ngrok.forward(addr=PORT, authtoken_from_env=True)

url = listener.url()

print(url)

> **IMPORTANT:** If your ngrok account is free, click on the link generated by the cell above to open a warning page. In this page, click on the *Visit Site* button. You need to do this to unblock the proxy connection.
>
> The button will redirect you to an error page that you can safely close.

Then, the asynchronous [start function](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.html#jchannel.start) instantiates a server, starts this server and returns it.

Notice that this function receives the tunnel URL as an argument.

In [ ]:
server = await jchannel.start(host=HOST, port=PORT, url=url)

## Opening a channel

A [server channel](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.channel.html#jchannel.channel.Channel) uses a server to call frontend JavaScript code from kernel Python code. The asynchronous [open method](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.server.html#jchannel.server.Server.open) instantiates a channel, opens this channel and returns it.

This method receives a string representing a JavaScript function. This function should receive a [client representation of the same channel](https://hashiprobr.github.io/jupyter-jchannel-client/Channel.html) and initialize it. The most important part of this initialization is setting the `handler` property to an object. The methods of this object define the API at the frontend.

The example below sets `handler` to an object with a single method called `indent`. This method uses the JavaScript [padStart method](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/String/padStart) to add blank spaces at the beginning of a string.

In [ ]:
channel = await server.open('''
    (channel) => {
        channel.handler = {
            indent(line, count) {
                return line.padStart(line.length + count);
            }
        };
    }
''')

And that's it! You can now call any method of the API from the notebook.

The `call` method returns an `asyncio.Task` that can be awaited for the result.

Arguments and return values can have any types that are JSON-serializable.

In [ ]:
await channel.call('indent', 'return', 4)

If the JavaScript method throws an exception, it is wrapped in a Python exception.

In [ ]:
await channel.call('indent', 4, 'return')